In [ ]:
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/data_utils.py
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/image_utils.py
!wget -q https://github.com/IntrotoML9103/Project/blob/main/final_utils.py

!wget -qO- https://github.com/DM-GY-9103-2024F-H/art-dataset/releases/latest/download/imgs.tar.gz | tar xz


In [ ]:
!ls

data_utils.py  final_utils.py  image_utils.py  imgs.zip  sample_data


In [ ]:
!unzip -q imgs.zip

In [ ]:
!rm -rf __MACOSX/

Data Processing

In [ ]:
import os
import random
from os import listdir, path
import torch
from torch import Tensor
from image_utils import make_image, open_image


random.seed(42)

artist_names = [
    "Albrecht_Dürer",
    "Alfred_Sisley",
    "Amedeo_Modigliani",
    "Andy_Warhol",
    "Diego_Velazquez",
    "Edgar_Degas",
    "Francisco_Goya",
    "Frida_Kahlo",
    "Giotto_di_Bondone",
    "Gustav_Klimt",
    "Henri_Matisse",
    "Hieronymus_Bosch",
    "Joan_Miro",
    "Kazimir_Malevich",
    "Leonardo_da_Vinci",
    "Marc_Chagall",
    "Mikhail_Vrubel",
    "Pablo_Picasso",
    "Paul_Gauguin",
    "Paul_Klee",
    "Peter_Paul_Rubens",
    "Pierre_Auguste_Renoir",
    "Pieter_Bruegel",
    "Raphael",
    "Rembrandt",
    "Rene_Magritte",
    "Salvador_Dali",
    "Sandro_Botticelli",
    "Titian",
    "Vincent_van_Gogh"
]

artists_dict = {name:index  for index, name in enumerate(artist_names)}

file_names = sorted([f for f in listdir("imgs") if f.endswith(".jpg")])

print(len(file_names))

random.shuffle(file_names)

train_files = file_names[:256]
test_files = file_names[-256:]
check_files = file_names[151:200]

print(len(train_files))
print(len(test_files))

image_dir = "imgs"
new_size = (224, 224)

train_list = []
train_label = [] # get name from file - artists_dict map the name to the number - e.g: index Durer - returns 0


test_list = []
test_label = []

check_list = []
check_label = []

count = 0;

for fname in train_files:
    a_name = "_".join(fname.split("_")[:-1])
    label_1 = artists_dict.get(a_name, -1)

    if label_1 != -1:
        img_space = path.join(image_dir, fname)
        new_img = open_image(img_space).resize(new_size)
        img_tr = Tensor(new_img.pixels).reshape(224,224,-1).permute(2,0,1)
        if img_tr.shape[0] == 1:
            img_tr = img_tr.repeat(3,1,1)

    train_list.append(img_tr)
    train_label.append(label_1)

2929
256
256


In [ ]:
for fname in test_files:
    a_name = "_".join(fname.split("_")[:-1])
    label = artists_dict.get(a_name, -1)

    if label != -1:
        img_space = path.join(image_dir, fname)
        new_img = open_image(img_space).resize(new_size)
        img_t = Tensor(new_img.pixels).reshape(224,224,-1).permute(2,0,1)
        if img_t.shape[0] == 1:
            img_t = img_t.repeat(3,1,1)

    test_list.append(img_t)
    test_label.append(label)

In [ ]:
from torch.utils.data import Dataset, DataLoader

x_train = torch.stack(train_list)
y_train = Tensor(train_label).long()

x_test = torch.stack(test_list)
y_test = Tensor(test_label).long()

class FaceDataset(Dataset):
  def __init__(self, imgs, labels):
    self.imgs = imgs
    self.labels = labels

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return [self.imgs[idx], self.labels[idx]]

train_dataloader = DataLoader(FaceDataset(x_train, y_train), batch_size=128) #shuffle=True)
test_dataloader = DataLoader(FaceDataset(x_test, y_test), batch_size=512)

In [ ]:
model = resnet34(weights=ResNet34_Weights.DEFAULT)
display(model)

In [ ]:
train_transforms = v2.Compose([
	v2.RandomHorizontalFlip(p=0.5),
	v2.RandomRotation(degrees=15),
	v2.RandomPerspective(distortion_scale=0.15, p=0.5),
])

eval_transforms = v2.Compose([
  v2.Resize(224),
  v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train, test = LFWUtils.train_test_split(0.5, resnet_loader=True, train_transform=train_transforms, eval_transform=eval_transforms)

In [ ]:
img,label = next(iter(train))
print(img.shape, img[0].shape)
display(v2.ToPILImage()(img[0]))

In [ ]:
mdevice = "cuda" if torch.cuda.is_available() else "cpu"

model.fc = nn.Linear(model.fc.in_features, len(LFWUtils.LABELS))
model = model.to(mdevice)

learning_rate = 5e-3
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

loss_fn = nn.CrossEntropyLoss()

img, label = next(iter(train))
out = model(img)

print("Input shape:", img.shape)
print("Output shape:", out.shape)
print("Parameters:", LFWUtils.count_parameters(model))

In [ ]:
for e in range(16):
  model.train()
  for imgs, labels in train:
    optim.zero_grad()
    labels_pred = model(imgs)
    loss = loss_fn(labels_pred, labels)
    loss.backward()
    optim.step()

  if e % 4 == 3:
    print(f"Epoch: {e} loss: {loss.item():.4f}")

In [ ]:
train_labels, train_predictions = LFWUtils.get_labels(model, train)
test_labels, test_predictions = LFWUtils.get_labels(model, test)
train_error = classification_error(train_labels, train_predictions)
test_error = classification_error(test_labels, test_predictions)
print(f"train error: {train_error:.4f}, test error: {test_error:.4f}")

display_confusion_matrix(train_labels, train_predictions, display_labels=LFWUtils.LABELS)
display_confusion_matrix(test_labels, test_predictions, display_labels=LFWUtils.LABELS)